In [ ]:
import sys
import os

sys.path.append(os.path.abspath("../tools"))
from envvars import load_env_vars_from_directory
import numpy as np
import pandas as pd

import requests
from urllib.parse import urljoin
from pprint import pprint
sys.path.append(os.path.abspath("../rag"))
from sttcast_rag_service import RelSearchRequest
import requests
from urllib.parse import urljoin
import json
import hmac
import hashlib
import time

In [ ]:
# Funciones de autenticación HMAC
def create_hmac_signature(secret_key: str, method: str, path: str, body: str, timestamp: str) -> str:
    """Crea una firma HMAC para autenticar la solicitud."""
    message = f"{method}|{path}|{body}|{timestamp}"
    return hmac.new(
        secret_key.encode(),
        message.encode(),
        hashlib.sha256
    ).hexdigest()

def create_auth_headers(secret_key: str, method: str, path: str, body: dict) -> dict:
    """Crea los headers de autenticación HMAC."""
    timestamp = str(int(time.time()))
    body_str = json.dumps(body, separators=(',', ':'), sort_keys=True)
    signature = create_hmac_signature(secret_key, method, path, body_str, timestamp)
    
    return {
        'X-Timestamp': timestamp,
        'X-Signature': signature,
        'X-Client-ID': 'testquery_notebook',
        'Content-Type': 'application/json'
    }

In [ ]:
# Variables de entorno
# Cargar variables de entorno desde archivos .env
load_env_vars_from_directory("../.env")

#
context_server_host = os.getenv('CONTEXT_SERVER_HOST')
if not context_server_host:
    raise ValueError("CONTEXT_SERVER_HOST environment variable is not set.")
context_server_port = int(os.getenv('CONTEXT_SERVER_PORT'))
if not context_server_port:
    raise ValueError("CONTEXT_SERVER_PORT environment variable is not set or invalid.")
context_server_url = f"http://{context_server_host}:{context_server_port}/"
if not context_server_url:
    raise ValueError("CONTEXT_SERVER_URL environment variable is not set.")
# rag_server_host = os.getenv('RAG_SERVER_HOST')
rag_server_host = "localhost"
if not rag_server_host:
    raise ValueError("RAG_SERVER_HOST environment variable is not set.")
rag_server_port = int(os.getenv('RAG_SERVER_PORT'))
if not rag_server_port:
    raise ValueError("RAG_SERVER_PORT environment variable is not set or invalid.")
rag_server_url = f"http://{rag_server_host}:{rag_server_port}/"
if not rag_server_url:
    raise ValueError("RAG_SERVER_URL environment variable is not set.")
# Cargar clave de autenticación para Context server
context_server_api_key = os.getenv('CONTEXT_SERVER_API_KEY')
if not context_server_api_key:
    raise ValueError("CONTEXT_SERVER_API_KEY environment variable is not set.")

In [21]:
get_context_path = "/getcontext"
get_context_url = urljoin(context_server_url, get_context_path)
get_context_url

'http://localhost:8001/getcontext'

In [22]:
query = "¿En qué episodios se han tratado redes neuronales no supervisadas?"

In [ ]:
payload = {
    "query": query,
    "n_fragments": 100
}

# Crear headers de autenticación HMAC
auth_headers = create_auth_headers(
    context_server_api_key,
    "POST",
    "/getcontext",
    payload
)

# ENVIAR EL JSON EXACTO QUE USAMOS PARA LA FIRMA
body_str = json.dumps(payload, separators=(',', ':'), sort_keys=True)
response = requests.post(get_context_url, data=body_str, headers=auth_headers)

print("Response status code:", response.status_code)
if response.status_code == 200:
    data = response.json()
    if 'context' in data:
        context = data['context']
        print("Context retrieved successfully:")
        pprint(context)
    else:
        print("No context found in the response.")
else:
    print("Failed to retrieve context. Status code:", response.status_code)
    print("Response content:", response.content)

Response status code: 200
Context retrieved successfully:
[{'content': 'No, y la red neuronal no tiene... Hablamos antes en otro '
             'desayuno, en otro coffee break, que la red neuronal lo que no '
             'tiene es la evolución humana, la evolución de especie. Nosotros '
             'en el cerebro no son conexiones aleatorias, son conexiones que '
             'han venido de millones de años de evolución. Eso lo tenemos en '
             'el cerebro porque cuando vemos una araña nos asustamos, eso no '
             'hay que aprenderlo desde pequeño, ya lo sabemos. Hay cosas que '
             'la red neuronal no ha evolucionado, no ha llegado por evolución. '
             'Supongo que eso será el futuro de las redes, que realmente '
             'aprenderán, no solo interpolarán.',
  'end': 1794.12,
  'epdate': '2017-12-21',
  'epname': 'ep142',
  'id': 8705,
  'start': 1764.23,
  'tag': 'Carlos Westendorp'},
 {'content': 'Me interesa más que el tema biofísico, que es

In [24]:
relsearch_path = "/relsearch"
relsearch_url = urljoin(rag_server_url, relsearch_path)
relsearch_url

'http://localhost:5500/relsearch'

In [25]:
relquery = RelSearchRequest(
    query=query,
    embeddings=context
).model_dump()
relquery

{'query': '¿En qué episodios se han tratado redes neuronales no supervisadas?',
 'embeddings': [{'tag': 'Carlos Westendorp',
   'epname': 'ep142',
   'epdate': '2017-12-21',
   'start': 1764.23,
   'end': 1794.12,
   'content': 'No, y la red neuronal no tiene... Hablamos antes en otro desayuno, en otro coffee break, que la red neuronal lo que no tiene es la evolución humana, la evolución de especie. Nosotros en el cerebro no son conexiones aleatorias, son conexiones que han venido de millones de años de evolución. Eso lo tenemos en el cerebro porque cuando vemos una araña nos asustamos, eso no hay que aprenderlo desde pequeño, ya lo sabemos. Hay cosas que la red neuronal no ha evolucionado, no ha llegado por evolución. Supongo que eso será el futuro de las redes, que realmente aprenderán, no solo interpolarán.'},
  {'tag': 'Héctor Socas',
   'epname': 'ep194',
   'epdate': '2018-12-20',
   'start': 4033.87,
   'end': 4055.04,
   'content': 'Me interesa más que el tema biofísico, que es

In [26]:
response = requests.post(relsearch_url, json=relquery)
print(response.ok)
json.loads(response.text)

True


{'tokens_prompt': 10966,
 'tokens_completion': 564,
 'tokens_total': 11530,
 'estimated_cost_usd': 0.001983,
 'search': {'es': "Las redes neuronales no supervisadas han sido un tema de discusión en varios episodios del podcast. En el episodio 194, Héctor Socas menciona la importancia de los algoritmos explicativos en redes neuronales, sugiriendo que algunas redes pueden aprender sin supervisión, aunque esto puede llevar a resultados difíciles de interpretar. Además, en el episodio 197, Sara Robisco habla sobre cómo ciertos algoritmos, como los genéticos, pueden producir resultados sin que se entienda completamente el proceso, lo que se relaciona con la naturaleza de las redes no supervisadas. En el episodio 138, se discute cómo las redes neuronales pueden ser consideradas 'cajas negras', lo que implica que su funcionamiento interno no siempre es claro, especialmente en modelos no supervisados. Estos episodios destacan la complejidad y los desafíos asociados con el uso de redes neuronal